In [1]:
import pandas as pd #Importing relevant library

In [ ]:
path = "/content/drive/MyDrive/Chicago1.csv" #Calling the file path from my drive
df = pd.read_csv(path)
df.head(5) #Taking a look at the dataset

In [3]:
def columns_to_drop(df): #Creating a function to drop redundant coloums that explain the same thing
  columns_to_drop=['Unnamed: 0', 'ID', 'Block', 'IUCR', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Year', 'Latitude', 'Longitude'] #List of columns to drop
  df=df.drop(columns=columns_to_drop, axis=1, inplace=True) #Setting coloums to drop and appling change to the whole dataset
  return df

columns_dropped = columns_to_drop(df)

In [ ]:
df.head() #Display the newly adjusted data

Using functions to create new columns for month, day and season

In [5]:
def add_date_features(df): #Creating a function to add new cloumns
    df['Date'] = pd.to_datetime(df['Date']) #Converting to datetime format
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day

    def get_season(month): #function to determine season based on month
        if month in [3, 4, 5]:
          return 'Spring'
        elif month in [6, 7, 8]:
          return 'Summer'
        elif month in [9, 10, 11]:
          return 'Autumn'
        else:
          return 'Winter'

    df['Season'] = df['Month'].apply(get_season) #creating season column
    return df


In [ ]:
add_date_features(df)

Use subsetting and grouping to show how the frequency of crime is distributed within Months, Days, Seasons with the most crime record and what crimes are highest and lowest?

In [7]:
def analyze_crime_frequency(df):
  #grouping by month, day and season
  monthly_crime_freq = df.groupby('Month')['Primary Type'].value_counts().reset_index(name='Frequency')
  daily_crime_freq = df.groupby('Day')['Primary Type'].value_counts().reset_index(name='Frequency')
  seasonal_crime_freq = df.groupby('Season')['Primary Type'].value_counts().reset_index(name='Frequency')

  #groups with the most crime records
  most_crime_month = monthly_crime_freq.loc[monthly_crime_freq.groupby('Month')['Frequency'].idxmax()]
  most_crime_day = daily_crime_freq.loc[daily_crime_freq.groupby('Day')['Frequency'].idxmax()]
  most_crime_season =seasonal_crime_freq.loc[seasonal_crime_freq.groupby('Season')['Frequency'].idxmax()]

  #highest and lowest crimes
  highest_crime = df['Primary Type'].value_counts().idxmax()
  lowest_crime = df['Primary Type'].value_counts().idxmin()

  return monthly_crime_freq, daily_crime_freq, seasonal_crime_freq, most_crime_month, most_crime_day, most_crime_season, highest_crime, lowest_crime

  print('Monthly Crime Frequency:\n',
        monthly_crime_freq)
  print('\nDaily Crime Frequency:\n',
        daily_crime_freq)
  print('\nSeasonal Crime Frequency:\n',
        seasonal_crime_freq)
  print('\nMost Crime Month:\n',
        most_crime_month)
  print('\nMost Crime Day:\n',
        most_crime_day)
  print('\nMost Crime Season:\n',
        most_crime_season)
  print('\nHighest Crime:\n',
        highest_crime)
  print('\nLowest Crime:\n',
        lowest_crime)

In [ ]:
analyze_crime_frequency(df)

According to location description, where crime happens most

---



In [ ]:
crime_location_freq = df['Location Description'].value_counts() #grouping by location description
most_crime_location = crime_location_freq.idxmax()

print('Location crime happens most:', most_crime_location)

Did the type of crime change as years go by?

In [ ]:
df['Year'] = df['Date'].dt.year #extracting year from date column
crime_yearly_freq = df.groupby(['Year', 'Primary Type']).size().unstack(fill_value=0)
crime_yearly_change = crime_yearly_freq.diff(axis=0)
types_changed = (crime_yearly_change != 0).any().any()

if types_changed:
  print('Yes, the type of crime changed over the years')
else:
  print('No, the type of crime did not change')

In [ ]:
#Showing this change on a line plot
import matplotlib.pyplot as plt

yearly_crime = df.groupby('Year').size()
plt.figure(figsize=(5,5)) #setting the figure size
yearly_crime.plot(kind='line', marker='o', color='b')
plt.title('Changes in Chicago Crime over the Years')
plt.xlabel('Year')
plt.ylabel('Number of Crimes')
plt.grid(True)#Adding grid lines
plt.show()